# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
import pymysql

Next, import `Orders` from Ironhack's database into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [2]:
# your code here
driver  = 'mysql+pymysql:'
user    = 'root'
password= 'Tarradellas90'
ip      = ':3306'
database= 'orders'

connection_string = f'{driver}//{user}:{password}@{ip}/{database}'
conn = create_engine(connection_string)

In [3]:
df_order = pd.read_sql('SELECT * FROM orders', conn)
original = df_order
df_order.head(5)

c:\users\migue granica\appdata\local\programs\python\python38-32\lib\site-packages\pymysql\cursors.py:170: Warning: (1366, "Incorrect string value: '\\xE1ndar ...' for column 'VARIABLE_VALUE' at row 485")
  result = self._query(query)


,index,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [4]:
for col in df_order.columns:
    print(col)
    uni = df_order[col].unique()
    if len(uni)<50:
        print(col)
        print('\t',uni)

index
InvoiceNo
StockCode
year
year
	 [2010 2011]
month
month
	 [12  1  2  3  4  5  6  7  8  9 10 11]
day
day
	 [3 4 5 7 1 2]
hour
hour
	 [ 8  9 10 11 12 13 14 15 16 17  7 18 19 20  6]
Description
Quantity
InvoiceDate
UnitPrice
CustomerID
Country
Country
	 ['United Kingdom' 'France' 'Australia' 'Netherlands' 'Germany' 'Norway'
 'EIRE' 'Switzerland' 'Spain' 'Poland' 'Portugal' 'Italy' 'Belgium'
 'Lithuania' 'Japan' 'Iceland' 'Channel Islands' 'Denmark' 'Cyprus'
 'Sweden' 'Finland' 'Austria' 'Greece' 'Singapore' 'Lebanon'
 'United Arab Emirates' 'Israel' 'Saudi Arabia' 'Czech Republic' 'Canada'
 'Unspecified' 'Brazil' 'USA' 'European Community' 'Bahrain' 'Malta' 'RSA']
amount_spent


In [5]:
#Sub Problem 1: How to aggregate the amount_spent for unique customers?
idx = df_order[['CustomerID','amount_spent']]
df_order_amount = idx.groupby('CustomerID').sum().sort_values('amount_spent', ascending=False)
df_order_amount.head(10)

,amount_spent
CustomerID,
14646,280206.02
18102,259657.30
17450,194550.79
16446,168472.50
14911,143825.06
12415,124914.53
14156,117379.63
17511,91062.38
16029,81024.84


In [6]:
#Sub Problem 2: How to select customers whose aggregated amount_spent is in a given quantile range?
customer_type = ['Casual', 'Usual', 'Preferred', 'VIP']
customers_type= pd.cut(df_order_amount['amount_spent'], len(customer_type), labels=customer_type, retbins= True)
customers_type


(CustomerID
 14646          VIP
 18102          VIP
 17450    Preferred
 16446    Preferred
 14911    Preferred
            ...    
 17956       Casual
 16454       Casual
 14792       Casual
 16738       Casual
 13256       Casual
 Name: amount_spent, Length: 4339, dtype: category
 Categories (4, object): [Casual < Usual < Preferred < VIP],
 array([  -280.20602,  70051.505  , 140103.01   , 210154.515  ,
        280206.02   ]))

In [7]:
customer_type = ['Casual', 'Usual', 'Preferred', 'VIP']
df_order_amount['Customer Cat']= pd.qcut(df_order_amount['amount_spent'], len(customer_type), labels = customer_type)
df_order_amount

,amount_spent,Customer Cat
CustomerID,,
14646,280206.02,VIP
18102,259657.30,VIP
17450,194550.79,VIP
16446,168472.50,VIP
14911,143825.06,VIP
...,...,...
17956,12.75,Casual
16454,6.90,Casual
14792,6.20,Casual


In [11]:
#w.loc[w.female != 'female', 'female'] = 0
cat_column = df_order_amount.drop(columns = ['amount_spent'])
cat_column

,Customer Cat
CustomerID,
14646,VIP
18102,VIP
17450,VIP
16446,VIP
14911,VIP
...,...
17956,Casual
16454,Casual
14792,Casual


In [12]:
df_order_label = pd.merge(df_order, cat_column, how='left', on = 'CustomerID')
df_order_label

,index,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent,Customer Cat
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30,VIP
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,VIP
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00,VIP
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,VIP
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,VIP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397919,397919,581587,22613,2011,12,5,12,pack of 20 spaceboy napkins,12,2011-12-09 12:50:00,0.85,12680,France,10.20,Preferred
397920,397920,581587,22899,2011,12,5,12,children s apron dolly girl,6,2011-12-09 12:50:00,2.10,12680,France,12.60,Preferred
397921,397921,581587,23254,2011,12,5,12,childrens cutlery dolly girl,4,2011-12-09 12:50:00,4.15,12680,France,16.60,Preferred
397922,397922,581587,23255,2011,12,5,12,childrens cutlery circus parade,4,2011-12-09 12:50:00,4.15,12680,France,16.60,Preferred


In [15]:
#Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?
df_customer_vip_preferred = df_order_label.loc[(df_order_label['Customer Cat'] == 'VIP') | (df_order_label['Customer Cat'] == 'Preferred')]
df_customer_vip_preferred.head(5)

,index,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent,Customer Cat
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30,VIP
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,VIP
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00,VIP
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,VIP
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,VIP


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [21]:
# your code here
df_customer_vip = df_order_label.loc[(df_order_label['Customer Cat'] == 'VIP')]
df_vips_by_country = df_customer_vip.groupby(['Country']).count()[['Customer Cat']]
df_vips_by_country

,Customer Cat
Country,
Australia,1028
Austria,158
Belgium,1557
Canada,135
Channel Islands,589
Cyprus,451
Denmark,217
EIRE,7238
Finland,504


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [20]:
# your code here
df_vips_preferred_by_country =  df_order_label.groupby(['Country']).count()[['Customer Cat']]
df_vips_preferred_by_country

,Customer Cat
Country,
Australia,1185
Austria,398
Bahrain,17
Belgium,2031
Brazil,32
Canada,151
Channel Islands,748
Cyprus,614
Czech Republic,25
